In [50]:
import pandas as pd

# Anomaly Detection

In [69]:
df = pd.read_json('anomaly_detection/results/aucs.json')
ranks = df.rank(axis=0, method='min', ascending=False)
mean_ranks = ranks.mean(axis=1)
df['mean_rank'] = mean_ranks
df.sort_values('mean_rank', inplace=True)
df

,nyc_taxi,ec2_request_latency_system_failure,msl,swat,smap,smd,mean_rank
patchtrad,0.9222,0.9987,0.6607,0.8446,0.6597,0.8693,3.000000
patchtst,0.6957,0.9989,0.6265,0.8426,0.6220,0.8818,4.333333
jepa,0.8482,0.9945,0.6304,0.8150,0.6887,0.8594,5.500000
patchad,0.9718,0.9984,0.6220,0.8225,0.6709,0.8176,6.000000
gat,0.6888,0.9989,0.6167,0.8156,0.6460,0.8197,6.666667
aelstm,0.6637,0.9985,0.6119,0.8402,0.6182,0.8280,8.000000
lstm,0.5107,0.9987,0.5950,0.8419,0.6039,0.8334,8.333333
tranad,0.5511,0.9672,0.6436,0.8149,0.5813,0.8835,8.500000
catch,NaN,NaN,0.6779,0.2375,0.5296,0.8941,9.000000
lstm_rev,0.6462,0.9984,0.6271,0.2383,0.5863,0.8582,9.333333


# Forecasting

In [72]:
df = pd.read_json('forecasting/results/mse.json')

ranks = df.rank(axis=0, method='min', ascending=True)
mean_ranks = ranks.mean(axis=1)
df['mean_rank'] = mean_ranks
df.sort_values('mean_rank', inplace=True)

df["encoder"]=" "
df["scratch"]=" "

for i in range(len(df)):
    index_name = df.index[i]
    index_name_split = index_name.split("_")
    if index_name_split[-1]=="True":
        df.at[index_name, "scratch"] = "True"
        df.at[index_name, "encoder"] = "trainable"
        df.index.values[i] = "PatchTST"
    else:
        if index_name_split[-2]=="True":
            df.at[index_name, "scratch"] = "False"
            df.at[index_name, "encoder"] = "freeze"
        else:
            df.at[index_name, "scratch"] = "False"
            df.at[index_name, "encoder"] = "trainable"
        df.index.values[i] = "JEPAtchTST"
df

,etth1,etth2,ettm1,ettm2,mean_rank,encoder,scratch
JEPAtchTST,0.457436,0.180563,0.411781,0.118213,1.50,freeze,False
JEPAtchTST,0.465032,0.188330,0.429293,0.116340,1.75,trainable,False
PatchTST,0.474271,0.226031,0.437121,0.117943,2.75,trainable,True
